<a href="https://colab.research.google.com/github/Shiksha-Yadav/WOC/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Normalization of dataset to ensure equal contribution of each feature
def normalizationx(df, min=None, max=None):
    if isinstance(df, pd.DataFrame):
      min_val = df.min().min() if min is None else min
      max_val = df.max().max() if max is None else max
    elif isinstance(df, np.ndarray):
      min_val = df.min() if min is None else min
      max_val = df.max() if max is None else max

    norm_df = (df - min_val) / (max_val - min_val)
    return norm_df, min_val, max_val


def normalizationy(df, min=None, max=None):
    min = df.min() if min is None else min
    max = df.max() if max is None else max
    norm_df = (df - min) / (max - min)
    return norm_df, min, max


# Denormalize to get values for test dataset
def denorm(normal_df, max_val, min_val):
    return (normal_df * (max_val - min_val) + min_val)


# Separate training and testing data
def matrix(df):
    x = df.iloc[1:, 1:-1].values  # all columns (features) except the last one (target)
    y = df.iloc[1:, -1].values  # last column (target)
    test_size = 0.2
    rows = x.shape[0]
    train_size = int(rows * (1 - test_size))
    x_train = x[:train_size]  # First 80% for training
    x_test = x[train_size:]  # Remaining 20% for testing
    y_train = y[:train_size]  # First 80% of target for training
    y_test = y[train_size:]  # Remaining 20% of target for testing
    return x_train, x_test, y_train, y_test


def find_gini(y):
    count = np.bincount(y)
    probabilities = count / len(y)
    gini = 1 - np.sum(probabilities**2)
    return gini


def best_split(x, y):
    best_gini = float("inf")
    best_split_ = None
    n_samples, n_features = x.shape
    for feature_index in range(n_features):
        feature_values = x[:, feature_index]
        thresholds = np.unique(feature_values)
        for threshold in thresholds:
            left_mask = feature_values <= threshold
            right_mask = ~left_mask
            left_y = y[left_mask]
            right_y = y[right_mask]
            if len(left_y) == 0 or len(right_y) == 0:
                continue
            gini_left = find_gini(left_y)
            gini_right = find_gini(right_y)
            weighted_gini = (len(left_y) / n_samples) * gini_left + (
                len(right_y) / n_samples
            ) * gini_right
            if weighted_gini < best_gini:
                best_gini = weighted_gini
                best_split_ = {
                    "feature_index": feature_index,
                    "threshold": threshold,
                    "left_data": (x[left_mask], left_y),
                    "right_data": (x[right_mask], right_y),
                }
    return best_split_


def tree(x, y, max_depth=None, depth=0):
    n_samples, n_features = x.shape
    unique_classes = np.unique(y)
    if len(unique_classes) == 1:
        return unique_classes[0]
    if max_depth is not None and depth >= max_depth:
        return np.bincount(y).argmax()
    best_split_ = best_split(x, y)
    if best_split_ is None:
        return np.bincount(y).argmax()
    left_tree = tree(
        best_split_["left_data"][0],
        best_split_["left_data"][1],
        max_depth,
        depth + 1,
    )
    right_tree = tree(
        best_split_["right_data"][0],
        best_split_["right_data"][1],
        max_depth,
        depth + 1,
    )
    return {
        "feature_index": best_split_["feature_index"],
        "threshold": best_split_["threshold"],
        "left": left_tree,
        "right": right_tree,
    }


def predict_row(row, tree):
    if isinstance(tree, dict):
        feature_value = row[tree["feature_index"]]
        if feature_value <= tree["threshold"]:
            return predict_row(row, tree["left"])
        else:
            return predict_row(row, tree["right"])
    else:
        return tree


def predict(x, tree):
    # Iterate directly over the rows of the NumPy array
    return [predict_row(row, tree) for row in x]


def f1_score(y_true, y_pred, num_classes):
    precision = np.zeros(num_classes)
    recall = np.zeros(num_classes)
    f1 = np.zeros(num_classes)
    for i in range(num_classes):
        tp = np.sum((y_pred == i) & (y_true == i))
        fp = np.sum((y_pred == i) & (y_true != i))
        fn = np.sum((y_pred != i) & (y_true == i))
        precision[i] = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall[i] = tp / (tp + fn) if (tp + fn) != 0 else 0
        f1[i] = (
            2 * (precision[i] * recall[i]) / (precision[i] + recall[i])
            if (precision[i] + recall[i]) != 0
            else 0
        )
    return np.mean(f1)


def plot_tree(tree, feature_names, depth=0):
    if isinstance(tree, dict):
        feature_name = feature_names[tree["feature_index"]]
        threshold = tree["threshold"]
        print(f"{'|  ' * depth}Feature: {feature_name} <= {threshold:.2f}")
        print(f"{'|  ' * depth}Left branch:")
        plot_tree(tree["left"], feature_names, depth + 1)
        print(f"{'|  ' * depth}Right branch:")
        plot_tree(tree["right"], feature_names, depth + 1)
    else:
        print(f"{'|  ' * depth}Leaf: Class {tree}")


if __name__ == "__main__":
    dataset = "multi_classification_train.csv"
    df1 = pd.read_csv(dataset)  # dataset
    x_train, x_test, y_train, y_test = matrix(df1)
    x_train_norm, min_x, max_x = normalizationx(x_train)
    x_test_norm, _, _ = normalizationx(x_test, min_x, max_x)
    # y_train_norm, min_y, max_y= normalizationy(y_train)
    # y_test_norm, _, _= normalizationy(y_test, min_y, max_y)
    tree_ = tree(x_train_norm, y_train, max_depth=3)
    y_pred = predict(x_test_norm, tree_)
    accuracy = np.mean(np.array(y_pred) == y_test)
    print(f"Accuracy: {accuracy:.4f}")
    print("\nDecision Tree Structure:")
    plot_tree(tree_, df1.columns[1:-1])
    num_classes = len(np.unique(y_train))  # Number of unique classes in target
    f1 = f1_score(y_test, y_pred, num_classes)
    print(f"F1 Score: {f1:.4f}")


Accuracy: 0.6465

Decision Tree Structure:
Feature: Feature_11 <= 0.49
Left branch:
|  Feature: Feature_17 <= 0.42
|  Left branch:
|  |  Feature: Feature_10 <= 0.33
|  |  Left branch:
|  |  |  Leaf: Class 3
|  |  Right branch:
|  |  |  Leaf: Class 2
|  Right branch:
|  |  Feature: Feature_10 <= 0.46
|  |  Left branch:
|  |  |  Leaf: Class 3
|  |  Right branch:
|  |  |  Leaf: Class 2
Right branch:
|  Feature: Feature_2 <= 0.45
|  Left branch:
|  |  Feature: Feature_3 <= 0.52
|  |  Left branch:
|  |  |  Leaf: Class 4
|  |  Right branch:
|  |  |  Leaf: Class 3
|  Right branch:
|  |  Feature: Feature_14 <= 0.36
|  |  Left branch:
|  |  |  Leaf: Class 3
|  |  Right branch:
|  |  |  Leaf: Class 1
F1 Score: 0.0000
